# Вебинар 2. Бейзлайны и детерминированные алгоритмы item-item

[Implicit](https://implicit.readthedocs.io/en/latest/quickstart.html) - очень быстрая и 
эффективная библиотека для рекоммендаций

Основные фичи:
    - Cython под капотом - высокая скорость
    - Множество приближенных алгоритмов - быстрее, чем оригинальные
    - Содежрит большинство популярных алгоритмов
    - Есть алгоритмы ранжирования
    - Поиск похожих товаров / юзеров
    - Есть возможность пересчета "холодного" юзера "на лету"
    - Возможность фильтровать товары при рекомендациях (Например, не рекомендовать женские товары мужчинам)
    - Есть метрики качества

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
data['week_no'].nunique()

95

In [4]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  2499
# items:  89051
# interactions:  2396804


In [5]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.describe()

,item_id,sales_value
count,8.905100e+04,89051.000000
mean,5.115772e+06,83.458481
std,5.178973e+06,1628.715079
min,2.567100e+04,0.000000
25%,9.665830e+05,3.500000
50%,1.448516e+06,10.780000
75%,9.553042e+06,46.105000
max,1.802456e+07,467993.620000


In [6]:
popularity = data.groupby('item_id')['user_id'].nunique().reset_index()
popularity.describe()

,item_id,user_id
count,8.905100e+04,89051.000000
mean,5.115772e+06,14.759767
std,5.178973e+06,45.904111
min,2.567100e+04,1.000000
25%,9.665830e+05,1.000000
50%,1.448516e+06,2.000000
75%,9.553042e+06,10.000000
max,1.802456e+07,2039.000000


**Note:**  
Еще есть данные по характеристикам товаров и пользователей. Они нам пригодятся через несколько вебинаров

In [7]:
item_features = pd.read_csv('/Users/annasmejoukha/Desktop/GeekBrains/rec_sys/webinar_2/product.csv')
item_features.head(2)

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [8]:
user_features = pd.read_csv('/Users/annasmejoukha/Desktop/GeekBrains/rec_sys/webinar_2/hh_demographic.csv')
user_features.head(2)

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


### Train-test split

В рекомендательных системах корректнее использовать train-test split по времени, а не случайно  
Я возьму последние 3 недели в качестве теста

In [9]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [10]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

# 1. Бейзлайны

Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)

In [11]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [12]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


### 1.1 Random recommendation

In [13]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [14]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

CPU times: user 2.32 s, sys: 8.08 ms, total: 2.33 s
Wall time: 2.33 s


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1008920, 1134633, 851468, 1282907, 1356008]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8293449, 2221431, 602245, 1104363, 1188129]"


### 1.2 Popularity-based recommendation

In [15]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [16]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: user 54.7 ms, sys: 2.18 ms, total: 56.9 ms
Wall time: 56.4 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1008920, 1134633, 851468, 1282907, 1356008]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8293449, 2221431, 602245, 1104363, 1188129]","[6534178, 6533889, 1029743, 6534166, 1082185]"


### 1.3 Weighted random recommender

- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

*Пример*  
item_1 - 5, item_2 - 7, item_3 - 4  # / sum  
item_1 - 5 / 16, item_2 - 7 / 16, item_3 - 4 / 16

In [17]:
# вес каждого товара - sum sales_value
item_sales = data_train.groupby('item_id')['sales_value'].sum()
item_sales

item_id
25671       20.94
26081        0.99
26093        1.59
26190        1.54
26355        1.98
            ...  
17381856     0.00
17382205     7.99
17383227     4.49
17827644     2.50
17829232    11.98
Name: sales_value, Length: 86865, dtype: float64

In [18]:
# уберем 0 для логарифмирования
item_sales.min()

0.0

In [19]:
# заменим 0 на близкое к 0 число
item_sales.values[item_sales.values==0] = 0.00000000001

In [20]:
log_weights = np.log(item_sales)
log_weights

item_id
25671        3.041661
26081       -0.010050
26093        0.463734
26190        0.431782
26355        0.683097
              ...    
17381856   -25.328436
17382205     2.078191
17383227     1.501853
17827644     0.916291
17829232     2.483239
Name: sales_value, Length: 86865, dtype: float64

In [21]:
#убираем отрицательные
log_weights = log_weights - log_weights.min()

In [22]:
log_weights.sum()

3226462.738864591

In [23]:
log_weights.min()

0.0

In [24]:
log_weights_norm = log_weights/log_weights.sum()

In [25]:
log_weights_norm.sum()

0.9999999999999998

In [26]:
log_weights_norm

item_id
25671       0.000012
26081       0.000011
26093       0.000011
26190       0.000011
26355       0.000011
              ...   
17381856    0.000003
17382205    0.000011
17383227    0.000011
17827644    0.000011
17829232    0.000012
Name: sales_value, Length: 86865, dtype: float64

In [27]:
def weighted_random_recommendation(items, weight_list, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items = np.array(items)
    recs = np.random.choice(items, size=n, p=weight_list, replace=False)
    
    
    return recs.tolist()

In [28]:
items = log_weights_norm.index
weights = log_weights_norm.values

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items, weights, n=5))

In [29]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1008920, 1134633, 851468, 1282907, 1356008]","[6534178, 6533889, 1029743, 6534166, 1082185]","[900023, 362395, 3028020, 12456847, 1090218]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8293449, 2221431, 602245, 1104363, 1188129]","[6534178, 6533889, 1029743, 6534166, 1082185]","[917858, 993648, 9336532, 10149428, 903308]"


### Выводы по бейзлайнам
- Фиксируют базовое качество;
- Бейзлайны могут быть фильтрами;
- Иногда бейзлайны лучше ML-модели

# 2. Детерминированные алгоритмы item-item

## 2.1 Item-Item Recommender / ItemKNN

In [30]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [31]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [32]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

/Users/annasmejoukha/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
user_item_matrix.shape

(2499, 5001)

In [34]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

In [35]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [36]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.31 s, sys: 408 ms, total: 1.72 s
Wall time: 596 ms


In [37]:
recs

[(2381, 78679.0),
 (3408, 72173.0),
 (2148, 57995.0),
 (3947, 17272.0),
 (3587, 14417.0)]

In [38]:
[id_to_itemid[rec[0]] for rec in recs]

[999999, 1082185, 981760, 1127831, 1098066]

In [39]:
%%time

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 52.7 ms, sys: 33.3 ms, total: 86 ms
Wall time: 38 ms


In [40]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1008920, 1134633, 851468, 1282907, 1356008]","[6534178, 6533889, 1029743, 6534166, 1082185]","[900023, 362395, 3028020, 12456847, 1090218]","[999999, 1082185, 981760, 1127831, 995242]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8293449, 2221431, 602245, 1104363, 1188129]","[6534178, 6533889, 1029743, 6534166, 1082185]","[917858, 993648, 9336532, 10149428, 903308]","[999999, 1082185, 981760, 1098066, 995242]"


### 4.2 Косинусное сходство и CosineRecommender

In [41]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.31 s, sys: 343 ms, total: 1.66 s
Wall time: 563 ms


In [42]:
[id_to_itemid[rec[0]] for rec in recs]

[1082185, 999999, 981760, 1127831, 1098066]

In [43]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 72.3 ms, sys: 60.4 ms, total: 133 ms
Wall time: 49 ms


In [44]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,cosine
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1008920, 1134633, 851468, 1282907, 1356008]","[6534178, 6533889, 1029743, 6534166, 1082185]","[900023, 362395, 3028020, 12456847, 1090218]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8293449, 2221431, 602245, 1104363, 1188129]","[6534178, 6533889, 1029743, 6534166, 1082185]","[917858, 993648, 9336532, 10149428, 903308]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]"


### 4.3 TF-IDF взвешивание и TFIDFRecommender

In [45]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.4 s, sys: 231 ms, total: 1.63 s
Wall time: 571 ms


In [46]:
[id_to_itemid[rec[0]] for rec in recs]

[1082185, 981760, 1127831, 999999, 1098066]

In [47]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])

CPU times: user 78.7 ms, sys: 70.6 ms, total: 149 ms
Wall time: 49.7 ms


In [48]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1008920, 1134633, 851468, 1282907, 1356008]","[6534178, 6533889, 1029743, 6534166, 1082185]","[900023, 362395, 3028020, 12456847, 1090218]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8293449, 2221431, 602245, 1104363, 1188129]","[6534178, 6533889, 1029743, 6534166, 1082185]","[917858, 993648, 9336532, 10149428, 903308]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]"


### 4.4 Трюк

In [49]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.25 s, sys: 338 ms, total: 1.58 s
Wall time: 527 ms


In [50]:
[id_to_itemid[rec[0]] for rec in recs]

[999999, 1082185, 1029743, 995785, 1004906]

In [51]:
%%time

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[999999], 
                                    recalculate_user=False)])

CPU times: user 53.8 ms, sys: 59.6 ms, total: 113 ms
Wall time: 29.9 ms


### 4.5 Измерим качество по precision@5

In [52]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1008920, 1134633, 851468, 1282907, 1356008]","[6534178, 6533889, 1029743, 6534166, 1082185]","[900023, 362395, 3028020, 12456847, 1090218]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8293449, 2221431, 602245, 1104363, 1188129]","[6534178, 6533889, 1029743, 6534166, 1082185]","[917858, 993648, 9336532, 10149428, 903308]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [53]:
result.to_csv('/Users/annasmejoukha/Desktop/GeekBrains/rec_sys/webinar_2/predictions_basic.csv', index=False)

In [54]:
pip install ipynb

Note: you may need to restart the kernel to use updated packages.


In [55]:
from ipynb.fs.full.my_metrics import precision_at_k

Можно ли улучшить бейзлайны, если считать их на топ-5000 товарах?

In [56]:
%%time

metrics = {}

for i in result.columns[1:]:
    metrics[i] = result.apply(lambda row: precision_at_k(row[i], row['actual'],5), axis=1).mean()

CPU times: user 440 ms, sys: 10.4 ms, total: 450 ms
Wall time: 449 ms


In [57]:
df = pd.DataFrame(metrics.values(), index = metrics.keys(), columns=['precision_at_k'])
df = df.sort_values(['precision_at_k'])
df

,precision_at_k
random_recommendation,0.000490
weighted_random_recommendation,0.000490
cosine,0.132909
itemitem,0.136925
tfidf,0.138981
popular_recommendation,0.155240
own_purchases,0.179693
actual,1.000000


* ### weighted_random_recommendation показал плохой результат по precision. Можно сделать вывод, что сэмплирование прямо пропорционально полулярности - малоэффективно, практически как и random_recommendation (0.000588 для weighted_random_recommendation и 0.000392 для random_recommendation)

## Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей K


### 1.1 Random recommendation top 5000

In [58]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [59]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1008920, 1134633, 851468, 1282907, 1356008]","[6534178, 6533889, 1029743, 6534166, 1082185]","[900023, 362395, 3028020, 12456847, 1090218]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8293449, 2221431, 602245, 1104363, 1188129]","[6534178, 6533889, 1029743, 6534166, 1082185]","[917858, 993648, 9336532, 10149428, 903308]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [60]:
items = data_train.item_id.unique()
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [61]:
result['random_recommendation_5'] = result['user_id'].apply(lambda x: random_recommendation(top_5000, n=5))
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,cosine,tfidf,own_purchases,random_recommendation_5
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1008920, 1134633, 851468, 1282907, 1356008]","[6534178, 6533889, 1029743, 6534166, 1082185]","[900023, 362395, 3028020, 12456847, 1090218]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[1090676, 1082627, 1046243, 1103105, 925437]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8293449, 2221431, 602245, 1104363, 1188129]","[6534178, 6533889, 1029743, 6534166, 1082185]","[917858, 993648, 9336532, 10149428, 903308]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[913623, 1034383, 987724, 12256520, 1003178]"


### 1.2 Popularity-based recommendation

In [62]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    return recs.tolist()


### 1.3 Weighted random recommender топ 5000

In [63]:
item_sales = data_train.groupby('item_id')['sales_value'].sum()
item_sales_sorted = item_sales.sort_values(axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last', 
                       ignore_index=False, key=None)
item_sales_5000 = item_sales_sorted.head(5000)
item_sales_5000

item_id
999999      2.997551e+06
6534178     4.477999e+05
6533889     4.048334e+04
1029743     3.576466e+04
6534166     3.017077e+04
                ...     
14043825    1.773000e+01
14050436    1.769000e+01
14043826    1.665000e+01
14043817    1.604000e+01
14050461    1.476000e+01
Name: sales_value, Length: 5000, dtype: float64

In [64]:
item_sales_5000.min()

14.760000000000003

In [65]:
# 0 отсутствуют
log_weights_5000 = np.log(item_sales_5000)
log_weights_5000

item_id
999999      14.913306
6534178     13.012102
6533889     10.608646
1029743     10.484716
6534166     10.314629
              ...    
14043825     2.875258
14050436     2.873000
14043826     2.812410
14043817     2.775086
14050461     2.691921
Name: sales_value, Length: 5000, dtype: float64

In [66]:
log_weights_5000_norm = log_weights_5000/log_weights_5000.sum()

In [67]:
log_weights_5000_norm.sum()

1.0

In [68]:
items_5000 = log_weights_5000_norm.index
weights_5000 = log_weights_5000_norm.values
result['weighted_random_recommendation_5К'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_5000, weights_5000, n=5))
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,cosine,tfidf,own_purchases,random_recommendation_5,weighted_random_recommendation_5К
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1008920, 1134633, 851468, 1282907, 1356008]","[6534178, 6533889, 1029743, 6534166, 1082185]","[900023, 362395, 3028020, 12456847, 1090218]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[1090676, 1082627, 1046243, 1103105, 925437]","[903609, 969866, 1130581, 887393, 970890]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8293449, 2221431, 602245, 1104363, 1188129]","[6534178, 6533889, 1029743, 6534166, 1082185]","[917858, 993648, 9336532, 10149428, 903308]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[913623, 1034383, 987724, 12256520, 1003178]","[12731714, 9392700, 971507, 932242, 9338009]"


In [69]:
%%time

metrics = {}

for i in result.columns[1:]:
    metrics[i] = result.apply(lambda row: precision_at_k(row[i], row['actual'],5), axis=1).mean()

CPU times: user 506 ms, sys: 1.51 ms, total: 507 ms
Wall time: 506 ms


In [70]:
df = pd.DataFrame(metrics.values(), index = metrics.keys(), columns=['precision_at_k'])
df = df.sort_values(['precision_at_k'])
df

,precision_at_k
random_recommendation,0.000490
weighted_random_recommendation,0.000490
random_recommendation_5,0.005583
weighted_random_recommendation_5К,0.006562
cosine,0.132909
itemitem,0.136925
tfidf,0.138981
popular_recommendation,0.155240
own_purchases,0.179693
actual,1.000000


## ItemItemRecommender, выбирая число соседей K.

In [71]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

/Users/annasmejoukha/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [72]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', 
                                  columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
user_item_matrix.shape

(2499, 5000)

In [74]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.338359343737495

In [75]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [76]:
user_item_matrix

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2498,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
%%time

for i in range(1, 11):

    model = ItemItemRecommender(K=i, num_threads=3) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)
    
    result['itemitem_' + str(i) + 'k'] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=[itemid_to_id[999999]], 
                                        recalculate_user=True)])

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

...

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

CPU times: user 12.2 s, sys: 1.47 s, total: 13.6 s
Wall time: 4.78 s


In [78]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,cosine,tfidf,own_purchases,random_recommendation_5,...,itemitem_1k,itemitem_2k,itemitem_3k,itemitem_4k,itemitem_5k,itemitem_6k,itemitem_7k,itemitem_8k,itemitem_9k,itemitem_10k
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1008920, 1134633, 851468, 1282907, 1356008]","[6534178, 6533889, 1029743, 6534166, 1082185]","[900023, 362395, 3028020, 12456847, 1090218]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[1090676, 1082627, 1046243, 1103105, 925437]",...,"[1082185, 1029743, 995785, 1004906, 1081177]","[1082185, 995242, 1029743, 840361, 904360]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 981760, 995242, 1127831, 840361]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 995242, 840361]","[1082185, 981760, 995242, 840361, 1127831]","[1082185, 981760, 995242, 840361, 1127831]","[1082185, 981760, 995242, 840361, 1127831]","[1082185, 981760, 995242, 840361, 1127831]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8293449, 2221431, 602245, 1104363, 1188129]","[6534178, 6533889, 1029743, 6534166, 1082185]","[917858, 993648, 9336532, 10149428, 903308]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[913623, 1034383, 987724, 12256520, 1003178]",...,"[1082185, 1098066, 6534178, 1127831, 1068719]","[1082185, 1098066, 6534178, 826249, 1127831]","[1082185, 981760, 1098066, 6534178, 826249]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 995242]"


In [79]:
metrics = {}

for i in result.columns[1:]:
    metrics[i] = result.apply(lambda row: precision_at_k(row[i], row['actual'],5), axis=1).mean()

/Users/annasmejoukha/Desktop/GeekBrains/rec_sys/webinar_2/my_metrics.ipynb:148: RuntimeWarning: invalid value encountered in long_scalars
  "source": [


In [80]:
df = pd.DataFrame(metrics.values(), index = metrics.keys(), columns=['precision_at_k'])
df = df.sort_values(['precision_at_k'])
df

,precision_at_k
random_recommendation,0.000490
weighted_random_recommendation,0.000490
random_recommendation_5,0.005583
weighted_random_recommendation_5К,0.006562
cosine,0.132909
itemitem,0.136925
tfidf,0.138981
itemitem_5k,0.151224
popular_recommendation,0.155240
itemitem_6k,0.155436


* ### Лучший результат по precision из всех рассмотренных алгоритмов получен для itemitem c K = 2 и itemitem c K = 2 ( 0.219344 и 0.218976 соответственно)